# How to build an advanced Chatbot with session memory using LangChain
* Advanced Chatbot LLM App.
    * Will be able to have a conversation.
    * Will remember previous interactions: will have memory.
    * Will be able to have different memories for different user sessions.
    * Will be able to remember a limited number of messages: limited memory.


## Concepts included
* Chat Model vs. LLM Model:
    *  Chat Model is based around messages.
    *  LLM Model is based around raw text.
* Chat History: allows Chat Model to remember previous interactions.

## Trick to avoid the nasty deprecation warnings from LangChain

In this exercise we will use the LangChain legacy chain LLMChain. It works well, but LangChain displays a nasty deprecation warning. To avoid it, we will enter the following code:

In [2]:
import warnings
from langchain._api import LangChainDeprecationWarning

warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [3]:
#!pip install python-dotenv
import os
from dotenv import load_dotenv, find_dotenv
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_groq import ChatGroq

In [5]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
chat = ChatGroq(
    groq_api_key = os.getenv("GROQ_API_KEY"),
    model_name = "llama3-70b-8192",
)

os.environ["LANGCHAIN_PROJECT"] = "Project Demo"

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [4]:
#!pip install langchain

## Connect with an LLM and start a conversation with it

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [5]:
#!pip install langchain-openai

* For this project, we will use OpenAI's gpt-3.5-turbo

In [6]:
from langchain_openai import ChatOpenAI

chatbot = ChatOpenAI(model="gpt-3.5-turbo")

* Human Message: the user input.

In [6]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favourite color is blue."),
]

#### Call the ChatModel (the LLM)

In [7]:
chat.invoke(messagesToTheChatbot)

AIMessage(content="Blue is a wonderful color! It's calming, soothing, and often associated with feelings of trust and reliability. Did you know that blue is also the most common favorite color among people? It's no wonder, since blue is the color of the sky and the ocean, which are two of the most beautiful natural wonders of our world!\n\nDo you have a specific shade of blue that you're particularly fond of? Like, maybe a bright cobalt blue or a softer, more muted blue like sky blue?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 16, 'total_tokens': 118, 'completion_time': 0.395609371, 'prompt_time': 0.000241866, 'queue_time': 0.055265224, 'total_time': 0.395851237}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None}, id='run--be590912-1473-42e3-87e2-b82b80a38a3d-0', usage_metadata={'input_tokens': 16, 'output_tokens': 102, 'total_tokens': 118})

#### Track the operation in LangSmith
* [Open LangSmith here](smith.langchain.com)

## Check if the Chatbot remembers your favorite color.

In [9]:
chat.invoke([
    HumanMessage(content="What is my favorite color?")
])

AIMessage(content="I'm just a language model, I don't have have access to personal information about you, so I don't know what your favorite color is.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 16, 'total_tokens': 47, 'completion_time': 0.105904729, 'prompt_time': 0.000215647, 'queue_time': 0.052193632000000004, 'total_time': 0.106120376}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None}, id='run--c944e8c3-e252-490b-a961-00d11d866bd4-0', usage_metadata={'input_tokens': 16, 'output_tokens': 31, 'total_tokens': 47})

* As you can see, our Chatbot cannot remember our previous interaction.

## Let's add memory to our Chatbot
* We will use the ChatMessageHistory package.
* We will save the Chatbot memory in a python dictionary called chatbotMemory.
* We will define the get_session_history function to create a session_id for each conversation.
* We will use the built-in runnable RunnableWithMesageHistory.

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [10]:
#!pip install langchain_community

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory



chatbotMemory = {}

# input: session_id, output: chatbotMemory[session_id]
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in chatbotMemory:
        chatbotMemory[session_id] = ChatMessageHistory()
    return chatbotMemory[session_id]


chatbot_with_message_history = RunnableWithMessageHistory(
    chat, 
    get_session_history
)

#### What is BaseChatMessageHistory and what it does?
BaseChatMessageHistory is what is called an **abstract base class** in Python. [See more info about this here](https://api.python.langchain.com/en/latest/chat_history/langchain_core.chat_history.BaseChatMessageHistory.html). This means it serves as a template or a foundational **blueprint for other classes**. It outlines a set of methods and structures that any class inheriting from it must implement or adhere to, but it cannot be used to create objects directly.

Here's a simpler breakdown of what it means for `BaseChatMessageHistory` to be an abstract base class:

1. **Blueprint for Other Classes:** It provides a predefined structure that other classes can follow. Think of it like an outline or a checklist for building something; it specifies what needs to be included, but it isn’t the final product.

2. **Cannot Create Instances:** You cannot create an instance of an abstract base class. Trying to create an object directly from `BaseChatMessageHistory` would result in an error because it's meant to be a guide, not something to use directly.

3. **Requires Implementation:** Any class that inherits from this abstract base class needs to implement specific methods outlined in `BaseChatMessageHistory`, such as methods for adding messages, retrieving messages, and clearing messages. The class sets the rules, and the subclasses need to follow these rules by providing the actual operational details.

4. **Purpose in Design:** Using an abstract base class helps ensure consistency and correctness in the implementation of classes that extend it. It's a way to enforce certain functionalities in any subclass, making sure that they all behave as expected without rewriting the same code multiple times.

Overall, the concept of an abstract base class is about setting standards and rules, while leaving the specific details of execution to be defined by subclasses that inherit from it.

#### Let's explain the previous code in simple terms
The previous code manages the chatbot's memory of conversations based on session identifiers. Here’s a breakdown of what the different components do:

1. **chatbotMemory**:
    - `chatbotMemory = {}`: This initializes an empty dictionary where session IDs and their respective chat histories will be stored.

2. **get_session_history Function**:
    - This function, `get_session_history`, takes a `session_id` as an argument and returns the chat history associated with that session.
    - If a chat history for the given `session_id` does not exist in `chatbotMemory`, a new instance of `ChatMessageHistory` is created and assigned to that `session_id` in the dictionary.
    - The function ensures that each session has its own unique chat history, stored and retrieved using the session ID.

3. **chatbot_with_message_history**:
    - `chatbot_with_message_history = RunnableWithMessageHistory(chatbot, get_session_history)`: This line creates an instance of `RunnableWithMessageHistory` using two arguments: `chatbot` and `get_session_history`.
    - The `chatbot` is passed along with the `get_session_history` function. This setup integrates the chatbot with the functionality to handle session-specific chat histories, allowing the chatbot to maintain continuity and context in conversations across different sessions.
    - **Learn more about RunnableWithMessageHistory** [here](https://python.langchain.com/v0.1/docs/expression_language/how_to/message_history/).

Overall, the code organizes and manages a chatbot's memory, enabling it to handle different sessions with users effectively by remembering previous messages within each session.

#### RunnableWithMessageHistory
**When invoking a new RunnableWithMessageHistory, we specify the corresponding chat history using a configurable parameter**. Let's say we want to create a chat memory for one user session, let's call it session1:

In [32]:
session1 = {"configurable": {"session_id": "001"}}

In [33]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite color is red.")],
    config=session1,
)

responseFromChatbot.content

'I remember! You told me earlier that your favorite color is red!'

In [34]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Easy one! Your favorite color is RED!'

## Let's now change the session_id and see what happens

Now let's create a chat memory for another user session, let's call it session2:

In [35]:
session2 = {"configurable": {"session_id": "002"}}

If the chatbot is using this new memory for session2, it will not be able to remember anything from the previous conversation in the session1:

In [36]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session2,
)

responseFromChatbot.content

"I still don't know! You haven't shared your favorite color with me yet!"

## Let's go back to session1 and see if the memory is still there

In [37]:
session1 = {"configurable": {"session_id": "001"}}

In [38]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'I know this one! Your favorite color is RED!'

As we can see, the chatbot is now able to remember the session1 conversation.

## Our ChatBot has session memory now. Let's check if it remembers the conversation from session2.

In [39]:
session2 = {"configurable": {"session_id": "002"}}

In [40]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My name is Mahima Shetty.")],
    config=session2,
)

responseFromChatbot.content

'I remember! Your name is indeed Mahima Shetty!'

In [41]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=session2,
)

responseFromChatbot.content

'Easy one! Your name is Mahima Shetty!'

In [42]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my favorite color?")],
    config=session2,
)

responseFromChatbot.content

"I still don't know! You haven't shared your favorite color with me yet, Mahima Shetty!"

## Our chatBot now remembers each of our conversations.

## The importance to manage the Conversation History
* The memory of a chatbot is included in the context window of the LLM so, if left unmanaged, can potentially overflow it.
* **We are now going to learn how to limit the size of the memory of a chatbot**.
* First, let's take a look at what is in the memory of our chatbot:

In [43]:
print(chatbotMemory)

{'001': InMemoryChatMessageHistory(messages=[HumanMessage(content='My favorite color is red.', additional_kwargs={}, response_metadata={}), AIMessage(content="Red is a fantastic choice! Red is a bold, energetic, and attention-grabbing emotions. It's often associated with passion, courage, and excitement. Did you know that red is also a stimulating color that can increase heart rate and energy levels?\n\nWhat do you think it is about the color red that resonates with you? Is there a particular shade or tone that you're especially drawn to?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 16, 'total_tokens': 98, 'completion_time': 0.278548933, 'prompt_time': 0.000221976, 'queue_time': 0.053199665, 'total_time': 0.278770909}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_dd4ae1c591', 'finish_reason': 'stop', 'logprobs': None}, id='run--c591b9b1-88e4-4a14-9362-943245fe1502-0', usage_metadata={'input_tokens': 16, 'output_token

* Now, **let's define a function to limit the number of messages stored in memory and add it to our chain with .assign**.

In [44]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_core.runnables import RunnablePassthrough


def limited_memory_of_messages(messages, number_of_messages_to_keep=2):
    return messages[-number_of_messages_to_keep:]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

limitedMemoryChain = (
    RunnablePassthrough.assign(messages=lambda x: limited_memory_of_messages(x["messages"]))
    | prompt 
    | chat
)

* The limited_memory_of_messages function allows you to trim the list of stored messages, keeping only a specified number of the latest ones. For example, if you have a long list of messages and you only want to keep the last two, this function will do that for you.
* The lambda function works in conjunction with the `limited_memory_of_messages` function. Here’s a simple breakdown:

    1. **Lambda Function**: The `lambda` keyword is used to create a small anonymous function in Python. The `lambda` function defined here takes one argument, `x`.

    2. **Function Argument**: The argument `x` is expected to be a dictionary that contains a key named `"messages"`. The value associated with this key is a list of messages.

    3. **Function Body**: The body of the `lambda` function calls the `limited_memory_of_messages` function. It passes the list of messages found in `x["messages"]` to this function.

    4. **Default Behavior of limited_memory_of_messages**: Since the `lambda` function does not specify the `number_of_messages_to_keep` parameter when it calls `limited_memory_of_messages`, the latter will default to keeping the last 2 messages from the list (as defined by the earlier function).

In essence, the `lambda` function is a shorthand way to apply the `limited_memory_of_messages` function to the message list contained within a dictionary. It automatically trims the list to the last two messages.

**Let's now create our new chatbot with limited message history**:

In [27]:
chatbot_with_limited_message_history = RunnableWithMessageHistory(
    limitedMemoryChain,
    get_session_history,
    input_messages_key="messages",
)

## Let's add 2 more messages to the session1 conversation:

In [28]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite vehicles are Vespa scooters.")],
    config=session1,
)

responseFromChatbot.content

'Vespa scooters are amazing! They\'re so stylish and iconic, aren\'t they? Vespa scooters evoke a sense of freedom and adventure, and they\'re perfect for cruising around town or exploring scenic routes.\n\nWhat is it about Vespa scooters that resonates with you? Is it their sleek design, their ease of use, or something else entirely?\n\n(By the way, I\'ve got a fun fact for you: Did you know that the name "Vespa" is a combination of the Italian word "vespa," which means "wasp," and the Italian pronunciation of the Piaggio company\'s name?)'

In [29]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite city is San Francisco.")],
    config=session1,
)

responseFromChatbot.content

"San Francisco is an amazing choice! San Francisco is a vibrant, eclectic, and iconic city that has something for everyone. From the Golden Gate Bridge to Alcatraz Island, Fisherman's Wharf, and steep hills with colorful Victorian houses, San Francisco is a treasure trove of sights, sounds, and flavors. And let's not forget the famous cable cars!\n\nWhat do you love most about San Francisco? Is it the city's laid-back vibe, its rich history, or something else entirely?\n\n(By the way, I've got a fun fact for you: Did you know that San Francisco is home to the oldest Chinatown in North America, with a rich history dating back to the mid-1800s?)"

## The chatbot memory has now 4 messages. Let's check the Chatbot with limited memory. 
* Remember, this chatbot only remembers the last 2 messages, so if we ask her about the first message she should not remember it.

In [31]:
responseFromChatbot = chatbot_with_limited_message_history.invoke(
    {
        "messages": [HumanMessage(content="what My favorite city?")],
    },
    config=session1,
)

responseFromChatbot.content

"You didn't mention a favorite city earlier. That's perfectly fine! If you don't have a favorite city, we can explore different cities and find one that might interest you.\n\nWould you like to learn about a new city or explore a particular type of city (e.g., beach city, mountain town, cultural hub)?"

* The chatbot with limited memory has behaved as we expected.

## Finally, let's compare the previous response with the one provided by the Chatbot with unlimited memory

In [45]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="what is my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'No question about it! Your favorite color is RED!'

* As you can see, this chatbot remembers our first message.

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 004-invoke-stream-batch.py
* In terminal, make sure you are in the directory of the file and run:
    * python 002-advanced-chatbot.py